1. use spearman correlation between OTUs and 5 VitD variables (with BH FDR corrected p-val <= 0.05 as threshold)
2. use lasso regression on all OTUs vs. 5 VitD variables (need Cross-validation to choose tuning parameter)

In [67]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests

## Spearman correlation

### merge biomtable with mapping file (output: dat)

In [20]:
mf = pd.read_csv('../data/mapping_cleaned_MrOS.txt', sep='\t', dtype=str, index_col='#SampleID')
bt = pd.read_csv('../data/biomtable.txt', sep='\t', dtype=str, index_col='#OTU ID')  
bt = bt.transpose()

In [21]:
print(mf.shape, bt.shape) # bt has an additional row of 'taxonomy'

(599, 66) (600, 4727)


In [22]:
mf.head()

,BarcodeSequence,LinkerPrimerSequence,Experiment_Design_Description,Library_Construction_Protocol,Linker,Platform,Center_Name,Center_Project,Instrument_Model,Title,...,OHV1D2,OHV1D2CT,OHVD2CT,OHVDTOT,OHV1DTOT,OHSEAS,VDstatus,Description,ratio_activation,ratio_catabolism
#SampleID,,,,,,,,,,,,,,,,,,,,,
BI0023,TCTGGTGACATT,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,NaN,NaN,25.8,39.3,3:SUMMER,sufficiency,Orwoll.BI0023.BI,1.5232558139534882,0.0686046511627907
BI0056,CAAGCATGCCTA,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,NaN,NaN,39.2,61.9,2:SPRING,sufficiency,Orwoll.BI0056.BI,1.579081632653061,0.09974489795918368
BI0131,CTATTTGCGACA,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,NaN,NaN,23.1,52.1,2:SPRING,sufficiency,Orwoll.BI0131.BI,2.2554112554112553,0.06450216450216449
BI0153,ATCGGCGTTACA,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,NaN,NaN,27.3,43.1,2:SPRING,sufficiency,Orwoll.BI0153.BI,1.5787545787545787,0.07838827838827839
BI0215,CCTCTCGTGATC,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,NaN,NaN,33.0,50.2,4:FALL,sufficiency,Orwoll.BI0215.BI,1.5212121212121212,0.1096969696969697


In [23]:
bt.head()

#OTU ID,TACGTAGGTGGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGTGTAGGCGGGATATCAAGTCAGAAGTGAAAATTACGGGCTCAACTCGTAACCTGCTTTTGAAACTGACATTCTTGAGTGAAGTAGAGGCAAGCGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTGCTGGGCTTTTACTGACGCTGAGGCTCGAAAGCGTGGGGAGCAAA,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGACGGGTCCTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGGGACCTTGAGTGCGGCAGAGGCAGGCGGAATTCGTGGTGTAGCGGTGAAATGCTTAGATATCACGAAGAACTCCGATTGCGAAGGCAGCTTGCTGGACCGTAACTGACGTTGATGCTCGAAAGTGCGGGTATCAAA,AACGTAGGTCACAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGGAAGACAAGTTGGAAGTGAAATCCATGGGCTCAACCCATGAACTGCTTTCAAAACTGTTTTTCTTGAGTAGTGCAGAGGTAGGCGGAATTCCCGGTGTAGCGGTGGAATGCGTAGATATCGGGAGGAACACCAGTGGCGAAGGCGGCCTACTGGGCACCAACTGACGCTGAGGCTCGAAAGTGTGGGTAGCAAA,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGATGGATGTTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGATATCTTGAGTGCAGTTGAGGCAGGCGGAATTCGTGGTGTAGCGGTGAAATGCTTAGATATCACGAAGAACTCCGATTGCGAAGGCAGCCTGCTAAGCTGCAACTGACATTGAGGCTCGAAAGTGTGGGTATCAAA,AACGTAGGTCACAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGGAAGACAAGTTGGAAGTGAAATCTATGGGCTCAACCCATAAACTGCTTTCAAAACTGTTTTTCTTGAGTAGTGCAGAGGTAGGCGGAATTCCCGGTGTAGCGGTGGAATGCGTAGATATCGGGAGGAACACCAGTGGCGAAGGCGGCCTACTGGGCACCAACTGACGCTGAGGCTCGAAAGTGTGGGTAGCAAA,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCGGACGCTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGGTGTCTTGAGTACAGTAGAGGCAGGCGGAATTCGTGGTGTAGCGGTGAAATGCTTAGATATCACGAAGAACTCCGATTGCGAAGGCAGCTTGCTGGACTGTAACTGACGCTGATGCTCGAAAGTGTGGGTATCAAA,TACGGAGGATTCAAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGTTTGATAAGTTAGAGGTGAAATTTCGGGGCTCAACCCTGAACGTGCCTCTAATACTGTTGAGCTAGAGAGTAGTTGCGGTAGGCGGAATGTATGGTGTAGCGGTGAAATGCTTAGAGATCATACAGAACACCGATTGCGAAGGCAGCTTACCAAACTATATCTGACGTTGAGGCACGAAAGCGTGGGGAGCAAA,TACGTATGGTGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGCAGGCGGTGCGGCAAGTCTGATGTGAAAGCCCGGGGCTCAACCCCGGTACTGCATTGGAAACTGTCGTACTAGAGTGTCGGAGGGGTAAGCGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTACTGGACGATAACTGACGCTGAGGCTCGAAAGCGTGGGGAGCAAA,TACGGAGGATCCAAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGTCAGATAAGTTAGAGGTGAAATTCCGGTGCTTAACACCGGCGCTGCCTCTAATACTGTTTGACTAGAGAGTAGTTGCGGTAGGCGGAATGTATGGTGTAGCGGTGAAATGCTTAGAGATCATACAGAACACCGATTGCGAAGGCAGCTTACCAAGCTATATCTGACGTTGAGGCACGAAAGCGTGGGGAGCAAA,TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGTGTAGGTGGCATCACAAGTCAGAAGTGAAAGCCCGGGGCTCAACCCCGGGACTGCTTTTGAAACTGTGGAGCTGGAGTGCAGGAGAGGCAAGTGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTGCTGGACTGTAACTGACACTGAGGCTCGAAAGCGTGGGGAGCAAA,...,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAAACTATGGGCTCAACCCATAAATTGCTTTCAAAACTGTTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGGTGTAGCGGTGGAATGCGTAGATATCGGGAGGAACACCAGTGGCGAAGGCGACCTACTGGGCACCAACTGACGCTGAGGCTCGAAAGCATGGGTAGCAAA,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGGTGGCAAGGCAAGTCAGATGTGAAAACCCGGGGCTCAACTCCGGGACTGCATTTGAAACTGTCTAGCTAGAGTGCAGGAGAGGTAAGTGGAATTCCTAGTGTAGCGGTGAAATGCATAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACACTGAGGCTCGAAAGCGTGGGGAGCAAA,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCGACGCAAGTCTGAAGTGAAATACCCGGGCTCAACCTGGGAACTGCTTTGGAAACTGTGTTGCTAGAGTGCTGGAGAGGTAAGCGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAAGAACACCAGTGGCGAAGGCGGCTTACTGGACAGTAACTGACGTTGAGGCTCGAAAACGTGGGGAGCAAA,TACGGAGGGTGCAAGCGTTAATCGGAATCACTGGGCGTAAAGCGCACGTAGGCGGCTTGGTAAGTCAGGGGTGAAATCCCACAGCCCAACTATGGAACTGCCTTTGATACTGCCAGGCTTGAGTACCGGAGAGGGTGGCGGAATTCCAGGTGTAGGAGTGAAATCCGTAGATATCTGGAGGAACACCGGTGGCGAAGGCGGCCACCTGGACGGTAACTGACGCTGAGGTGCGAAAGCGTGGGTAGCAAA,TACGTAGGTGGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGTGCGCAGGCGGGATTGCAAGTTGGATGTGAAATACCGGGGCTTAACCCCGGAGCTGCATCCAAAACTGTAGTTCTTGAGTGGAGTAGAGGTAAGCGGAATTCCGAGTGTAGCGGTGAAATGCGTAGATATTCGGAGGAACACCAGCGGCGAAGGCGGCTTACTGGGCTCTAACTGACGCTGAGGCACGAAAGCATGGGTAGCAAA,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGCAGGCGGTCTTGTAAGTCCGGTGTGAAAACCCGAGGCTCAACCTCGGGACTGCACTGGAAACTATGGGACTGGAGTGTCGGAGAGGTAAGTGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTACTGGACGACAACTGACGCTGAGGCTCGAAAGCG

In [25]:
dat = pd.merge(mf, bt, left_index=True, right_index=True)

In [26]:
dat.shape

(599, 4793)

In [27]:
dat.head()

,BarcodeSequence,LinkerPrimerSequence,Experiment_Design_Description,Library_Construction_Protocol,Linker,Platform,Center_Name,Center_Project,Instrument_Model,Title,...,AACGTAGGGTGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGCAGGCGGACCGGCAAGTTGGAAGTGAAAACTATGGGCTCAACCCATAAATTGCTTTCAAAACTGTTGGCCTTGAGTAGTGCAGAGGTAGGTGGAATTCCCGGTGTAGCGGTGGAATGCGTAGATATCGGGAGGAACACCAGTGGCGAAGGCGACCTACTGGGCACCAACTGACGCTGAGGCTCGAAAGCATGGGTAGCAAA,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGGTGGCAAGGCAAGTCAGATGTGAAAACCCGGGGCTCAACTCCGGGACTGCATTTGAAACTGTCTAGCTAGAGTGCAGGAGAGGTAAGTGGAATTCCTAGTGTAGCGGTGAAATGCATAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACACTGAGGCTCGAAAGCGTGGGGAGCAAA,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCGACGCAAGTCTGAAGTGAAATACCCGGGCTCAACCTGGGAACTGCTTTGGAAACTGTGTTGCTAGAGTGCTGGAGAGGTAAGCGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAAGAACACCAGTGGCGAAGGCGGCTTACTGGACAGTAACTGACGTTGAGGCTCGAAAACGTGGGGAGCAAA,TACGGAGGGTGCAAGCGTTAATCGGAATCACTGGGCGTAAAGCGCACGTAGGCGGCTTGGTAAGTCAGGGGTGAAATCCCACAGCCCAACTATGGAACTGCCTTTGATACTGCCAGGCTTGAGTACCGGAGAGGGTGGCGGAATTCCAGGTGTAGGAGTGAAATCCGTAGATATCTGGAGGAACACCGGTGGCGAAGGCGGCCACCTGGACGGTAACTGACGCTGAGGTGCGAAAGCGTGGGTAGCAAA,TACGTAGGTGGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGTGCGCAGGCGGGATTGCAAGTTGGATGTGAAATACCGGGGCTTAACCCCGGAGCTGCATCCAAAACTGTAGTTCTTGAGTGGAGTAGAGGTAAGCGGAATTCCGAGTGTAGCGGTGAAATGCGTAGATATTCGGAGGAACACCAGCGGCGAAGGCGGCTTACTGGGCTCTAACTGACGCTGAGGCACGAAAGCATGGGTAGCAAA,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGCAGGCGGTCTTGTAAGTCCGGTGTGAAAACCCGAGGCTCAACCTCGGGACTGCACTGGAAACTATGGGACTGGAGTGTCGGAGAGGTAAGTGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTACTGGACGACAACTGACGCTGAGGCTCGAAAGCGTGGGGAGCAAA,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGGGATTTAAGTCAGCGGTGAAAGTTTGAGGCTCAACCTTAAAATTGCCGTTGAAACTGGGTTTCTTGAGTGTATTTGAAGTAGGCGGAATTCGTTGTGTAGCGGTGAAATGCATAGATATAACGAAGAACTCCAATTGCGCAGGCAGCTTACTAAACTACAACTGACGCTGAAGCACGAAAGCGTGGGTATCAAA,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCGAAGCAAGTCAGAAGTGAAAACCCCGGGCTTAACCTGGGGACTGCTTTTGAAACTGTTTGGCTGGAGTGCAGGAGAGGTGAGTGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACGTTGAGGCTCGAAAGCGTGGGGAGCAAA,TACGTAGGTGGCAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAGCGTAGGCGGGGAGACAAGTTGAATGTCTAAACTATCGGCTTAACTGATAGTCGCGTTCAAAACTATCACTCTTGAGTGCAGTAGAGGTAGGCAGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCCTACTGGGCTGTAACTGACGCTGAGGCTCGAAAGCGTGGGTAGCAAAC,TACGTAGGGGGCAAGAGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCATGACAAGCCAGATGTGAAAACCCAGGGCTCAACCCTGGGACTGCATTTGGAACTGCCAGGCTGGAGTGCAGGAGAGGTAAGCGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACGTTGAGGCTCGAAAGCGTGGGGAGCAAA
BI0023,TCTGGTGACATT,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BI0056,CAAGCATGCCTA,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BI0131,CTATTTGCGACA,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BI0153,ATCGGCGTTACA,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BI0215,CCTCTCGTGATC,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D...,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,MrOS_VitaminD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
vars_vd = np.array(['OHV1D3', 'OHVD3', 'OHV24D3', 'ratio_activation', 'ratio_catabolism'])
dat[vars_vd] = dat[vars_vd].apply(pd.to_numeric, errors='coerce')

In [40]:
dat[vars_vd].describe()

,OHV1D3,OHVD3,OHV24D3,ratio_activation,ratio_catabolism
count,567.000000,556.000000,567.000000,556.000000,556.000000
mean,57.774956,35.229137,3.430864,1.772433,0.094776
std,19.772731,12.450758,1.834771,0.735064,0.029770
min,10.700000,7.800000,0.300000,0.398077,0.018788
25%,44.100000,27.400000,2.175000,1.315582,0.074216
50%,55.500000,33.650000,3.180000,1.659602,0.092821
75%,66.300000,41.825000,4.235000,2.080741,0.112849
max,156.000000,104.000000,14.070000,6.727273,0.197786


In [57]:
otu_cols = dat.columns[mf.shape[1]:dat.shape[1]]
len(otu_cols)

4727

### correlation

In [ ]:
results_spearman = []
for i in range(len(vars_vd)):
    #results_spearman = []
    for j in range(len(otu_cols)):
        tmp = dat[[vars_vd[i], otu_cols[j]]].dropna(axis=0, how='any')
        rho, pval = spearmanr(tmp[vars_vd[i]], tmp[otu_cols[j]])
        tax = bt.loc['taxonomy'][otu_cols[j]]
        results_spearman.append([vars_vd[i], otu_cols[j], tax, rho, pval])

results_spearman = pd.DataFrame(results_spearman, columns=['vars', 'otu', 'taxonomy', 'rho', 
                                                           'pval']).dropna(axis=0, how='any')
# BH FDR correction
results_spearman['fdr corrected pval'] = multipletests(results_spearman['pval'], method = 'fdr_bh')[1]
results_spearman = results_spearman.sort_values(['rho'], ascending=False)

/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3162: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3163: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x)

In [ ]:
results.head(10)

## Lasso Regression

In [81]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

In [82]:
vars_vd = np.array(['OHV1D3', 'OHVD3', 'OHV24D3', 'ratio_activation', 'ratio_catabolism'])
otu_cols = dat.columns[64:4790]

In [89]:
# drop all missing values
tmp = dat[np.append(vars_vd, otu_cols)]
tmp.head()

,OHV1D3,OHVD3,OHV24D3,ratio_activation,ratio_catabolism,0,1,2,3,4,...,4716,4717,4718,4719,4720,4721,4722,4723,4724,4725
BI0023,39.3,25.8,1.77,1.523256,0.068605,0.0,0.0,223.0,4071.0,313.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BI0056,61.9,39.2,3.91,1.579082,0.099745,88.0,0.0,120.0,5639.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BI0131,52.1,23.1,1.49,2.255411,0.064502,26.0,0.0,60.0,257.0,404.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BI0153,43.1,27.3,2.14,1.578755,0.078388,0.0,0.0,236.0,193.0,1984.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BI0215,50.2,33.0,3.62,1.521212,0.109697,0.0,0.0,242.0,685.0,735.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
scaler = StandardScaler()
small= tmp[np.append(vars_vd[0], otu_cols)].dropna(axis=0, how='any')
X = scaler.fit_transform(small[otu_cols])
Y = small[vars_vd[0]]
names = otu_cols

In [91]:
# Create a function called lasso,
def lasso(alphas):
    '''
    Takes in a list of alphas. Outputs a dataframe containing the coefficients of lasso regressions from each alpha.
    '''
    # Create an empty data frame
    df = pd.DataFrame()

    # Create a column of feature names
    df['OTU names'] = names

    # For each alpha value in the list of alpha values,
    for alpha in alphas:
        # Create a lasso regression with that alpha value,
        lasso = Lasso(alpha=alpha)

        # Fit the lasso regression
        lasso.fit(X, Y)

        # Create a column name for that alpha value
        column_name = 'Alpha = %f' % alpha

        # Create a column of coefficient values
        df[column_name] = lasso.coef_

    # Return the datafram    
    return df

In [98]:
#lasso([.0001, .5, 10])

/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


,OTU names,Alpha = 0.000100,Alpha = 0.500000,Alpha = 10.000000
0,0,1.133689,0.0,0.0
1,1,0.706943,0.0,0.0
2,2,0.254905,-0.0,0.0
3,3,-1.028152,-0.0,-0.0
4,4,-0.348684,-0.0,-0.0


In [111]:
table = lasso([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

In [112]:
table.head()

,OTU names,Alpha = 0.500000,Alpha = 1.000000,Alpha = 1.500000,Alpha = 2.000000,Alpha = 2.500000,Alpha = 3.000000,Alpha = 3.500000,Alpha = 4.000000,Alpha = 4.500000,Alpha = 5.000000
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
4,4,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [119]:
table.loc[table['Alpha = 3.000000'] != 0]

,OTU names,Alpha = 0.500000,Alpha = 1.000000,Alpha = 1.500000,Alpha = 2.000000,Alpha = 2.500000,Alpha = 3.000000,Alpha = 3.500000,Alpha = 4.000000,Alpha = 4.500000,Alpha = 5.000000
63,63,1.271302,1.042666,0.900248,0.875052,0.739639,0.485838,0.185897,0.000000,0.0,0.0
131,131,-1.266139,-1.404517,-1.445757,-1.305114,-1.027894,-0.699039,-0.345538,-0.000000,-0.0,-0.0
188,188,-0.926989,-0.877781,-0.843169,-0.734756,-0.461080,-0.158719,-0.000000,-0.000000,-0.0,-0.0
237,237,0.000000,0.000000,0.019175,0.242775,0.400080,0.332189,0.214622,0.000000,0.0,0.0
384,384,1.096472,1.205417,1.093996,1.024922,0.769250,0.505870,0.191158,0.000000,0.0,0.0
505,505,0.000000,0.099423,0.406352,0.570179,0.518540,0.286213,0.000000,0.000000,0.0,0.0
2116,2116,1.506435,1.666369,1.580376,1.157234,0.808036,0.504874,0.207639,0.000000,0.0,0.0
2545,2545,2.174811,1.994652,1.536085,1.368677,1.110300,0.801855,0.488216,0.140397,0.0,0.0
3484,3484,2.459450,2.093759,1.716113,1.307467,0.847264,0.444003,0.005585,0.000000,0.0,0.0
3598,3598,2.472380,2.242808,1.795477,1.250062,0.711844,0.172295,0.000000,0.000000,0.0,0.0


In [123]:
list = np.array([63, 131, 188, 237, 384, 505, 2116, 2545, 3484, 3598])
for i in range(len(list)):
    print(bt.loc['taxonomy'][list[i]])

k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__Oscillospira; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae; g__; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__; g__; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae; g__; s__
k__Bacteria; p__Cyanobacteria; c__4C0d-2; o__YS2; f__; g__; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae; g__Lachnospira; s__
k__Bacteria; p__Proteobacteria; c__Deltaproteobacteria; o__Desulfovibrionales; f__Desulfovibrionaceae; g__Bilophila; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae; g__Lachnospira; s__


In [124]:
scaler = StandardScaler()
small= tmp[np.append(vars_vd[1], otu_cols)].dropna(axis=0, how='any')
X = scaler.fit_transform(small[otu_cols])
Y = small[vars_vd[1]]
names = otu_cols

In [125]:
table = lasso([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

In [126]:
table.loc[table['Alpha = 3.000000'] != 0]

,OTU names,Alpha = 0.500000,Alpha = 1.000000,Alpha = 1.500000,Alpha = 2.000000,Alpha = 2.500000,Alpha = 3.000000,Alpha = 3.500000,Alpha = 4.000000,Alpha = 4.500000,Alpha = 5.000000
3323,3323,1.359633,1.095938,1.034084,0.742471,0.446554,0.034751,0.0,0.0,0.0,0.0


In [127]:
bt.loc['taxonomy'][3323]

'k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__; s__'

In [128]:
scaler = StandardScaler()
small= tmp[np.append(vars_vd[2], otu_cols)].dropna(axis=0, how='any')
X = scaler.fit_transform(small[otu_cols])
Y = small[vars_vd[2]]
names = otu_cols
table = lasso([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

In [130]:
table.loc[table['Alpha = 2.000000'] != 0]

,OTU names,Alpha = 0.500000,Alpha = 1.000000,Alpha = 1.500000,Alpha = 2.000000,Alpha = 2.500000,Alpha = 3.000000,Alpha = 3.500000,Alpha = 4.000000,Alpha = 4.500000,Alpha = 5.000000


In [131]:
scaler = StandardScaler()
small= tmp[np.append(vars_vd[3], otu_cols)].dropna(axis=0, how='any')
X = scaler.fit_transform(small[otu_cols])
Y = small[vars_vd[3]]
names = otu_cols
table = lasso([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

In [133]:
table.loc[table['Alpha = 2.000000'] != 0]

,OTU names,Alpha = 0.500000,Alpha = 1.000000,Alpha = 1.500000,Alpha = 2.000000,Alpha = 2.500000,Alpha = 3.000000,Alpha = 3.500000,Alpha = 4.000000,Alpha = 4.500000,Alpha = 5.000000


In [134]:
scaler = StandardScaler()
small= tmp[np.append(vars_vd[3], otu_cols)].dropna(axis=0, how='any')
X = scaler.fit_transform(small[otu_cols])
Y = small[vars_vd[3]]
names = otu_cols
table = lasso([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

In [137]:
table.loc[table['Alpha = 1.000000'] != 0]

,OTU names,Alpha = 0.500000,Alpha = 1.000000,Alpha = 1.500000,Alpha = 2.000000,Alpha = 2.500000,Alpha = 3.000000,Alpha = 3.500000,Alpha = 4.000000,Alpha = 4.500000,Alpha = 5.000000


In [140]:
scaler = StandardScaler()
small= tmp[np.append(vars_vd[4], otu_cols)].dropna(axis=0, how='any')
X = scaler.fit_transform(small[otu_cols])
Y = small[vars_vd[4]]
names = otu_cols
table = lasso([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

In [141]:
table.loc[table['Alpha = 1.000000'] != 0]

,OTU names,Alpha = 0.500000,Alpha = 1.000000,Alpha = 1.500000,Alpha = 2.000000,Alpha = 2.500000,Alpha = 3.000000,Alpha = 3.500000,Alpha = 4.000000,Alpha = 4.500000,Alpha = 5.000000


In [100]:
clf = Lasso(alpha=0.5)
clf.fit(X, Y)
Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [101]:
print(clf.coef_)

[ 0.  0. -0. ...,  0.  0.  0.]


In [103]:
sum(np.abs(clf.coef_))

139.64256393773678

In [102]:
print(clf.intercept_)

57.7749559083


## correlation with OHV1D3

In [34]:
vars_vd = np.array(['OHV1D3', 'OHVD3', 'OHV24D3', 'ratio_activation', 'ratio_catabolism'])
otu_cols = dat.columns[64:4790]
results_spearman = []
results_pearson = []
for i in range(len(otu_cols)):
    tmp = dat[['OHV1D3', otu_cols[i]]].dropna(axis=0, how='any')
    rho1, pval1 = spearmanr(tmp['OHV1D3'], tmp[otu_cols[i]])
    tax = bt.loc['taxonomy'][otu_cols[i]]
    results_spearman.append([vars_vd[1], otu_cols[i], tax, rho1, pval1])
#     rho2, pval2 = pearsonr(tmp['OHV1D3'], tmp[otu_cols[i]]) 
#     results_pearson.append([otu_cols[i], vars_vd[1], rho2, pval2]) error
results_spearman = pd.DataFrame(results_spearman, columns=['vars', 'otu', 'taxonomy', 'rho', 'pval']).dropna(axis=0, how='any')

/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3162: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3163: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x)

In [37]:
# pvalue after BH correction
from statsmodels.sandbox.stats.multicomp import multipletests
results = results_spearman
results['fdr corrected pval'] = multipletests(results['pval'], method = 'fdr_bh')[1]

In [38]:
results = results.sort_values(['rho'], ascending=False)
results.head(10)

,vars,otu,taxonomy,rho,pval,fdr corrected pval
289,OHVD3,289,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.176235,0.000024,0.028449
237,OHVD3,237,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.175983,0.000025,0.028449
0,OHVD3,0,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.175859,0.000025,0.028449
341,OHVD3,341,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.174129,0.000031,0.028449
315,OHVD3,315,k__Bacteria; p__Tenericutes; c__RF3; o__ML615J...,0.172459,0.000037,0.028449
62,OHVD3,62,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.170019,0.000047,0.031502
524,OHVD3,524,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.165885,0.000072,0.042197
389,OHVD3,389,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.162297,0.000104,0.048471
346,OHVD3,346,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.162295,0.000104,0.048471
603,OHVD3,603,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.155101,0.000209,0.087486


In [40]:
results.taxonomy[1]

'k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__'

In [43]:
for i in range(9):
    print(results.taxonomy[i])

k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__Ruminococcus; s__
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__Faecalibacterium; s__prausnitzii
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__Faecalibacterium; s__prausnitzii
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__uniformis
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__; g__; s__
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Rikenellaceae; g__; s__


In [ ]:
## significant bacteria with OHVD3
Ruminococcus, Bacteroides, Faecalibacterium, Firmicutes

## correlation with OHVD3

In [44]:
vars_vd = np.array(['OHV1D3', 'OHVD3', 'OHV24D3', 'ratio_activation', 'ratio_catabolism'])
otu_cols = dat.columns[64:4790]
results_spearman = []
results_pearson = []
for i in range(len(otu_cols)):
    tmp = dat[['OHVD3', otu_cols[i]]].dropna(axis=0, how='any')
    rho1, pval1 = spearmanr(tmp['OHVD3'], tmp[otu_cols[i]])
    tax = bt.loc['taxonomy'][otu_cols[i]]
    results_spearman.append([vars_vd[1], otu_cols[i], tax, rho1, pval1])

/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3162: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3163: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x)

In [45]:
results_spearman = pd.DataFrame(results_spearman, columns=['vars', 'otu', 'taxonomy', 'rho', 'pval']).dropna(axis=0, how='any')
results = results_spearman
results['fdr corrected pval'] = multipletests(results['pval'], method = 'fdr_bh')[1]
results = results.sort_values(['rho'], ascending=False)
results.head(10) # same fdr p-value for all?

,vars,otu,taxonomy,rho,pval,fdr corrected pval
1709,OHVD3,1709,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.147461,0.000486,0.826063
2061,OHVD3,2061,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.142533,0.000750,0.826063
557,OHVD3,557,k__Bacteria; p__Firmicutes; c__Erysipelotrichi...,0.140299,0.000909,0.826063
1023,OHVD3,1023,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.127120,0.002674,0.826063
2703,OHVD3,2703,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.126345,0.002841,0.826063
353,OHVD3,353,k__Bacteria; p__Firmicutes; c__Erysipelotrichi...,0.120728,0.004362,0.826063
1622,OHVD3,1622,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.120525,0.004428,0.826063
3663,OHVD3,3663,k__Bacteria; p__Proteobacteria; c__Alphaproteo...,0.114437,0.006909,0.826063
2968,OHVD3,2968,k__Bacteria; p__Bacteroidetes; c__Bacteroidia;...,0.114426,0.006915,0.826063
851,OHVD3,851,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.114389,0.006933,0.826063


## correlation with OHV24D3

In [51]:
vars_vd = np.array(['OHV1D3', 'OHVD3', 'OHV24D3', 'ratio_activation', 'ratio_catabolism'])
otu_cols = dat.columns[64:4790]
results_spearman = []
results_pearson = []
for i in range(len(otu_cols)):
    tmp = dat[['OHV24D3', otu_cols[i]]].dropna(axis=0, how='any')
    rho1, pval1 = spearmanr(tmp['OHV24D3'], tmp[otu_cols[i]])
    tax = bt.loc['taxonomy'][otu_cols[i]]
    results_spearman.append([vars_vd[1], otu_cols[i], tax, rho1, pval1])

/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3162: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3163: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x)

In [52]:
results_spearman = pd.DataFrame(results_spearman, columns=['vars', 'otu', 'taxonomy', 'rho', 'pval']).dropna(axis=0, how='any')
results = results_spearman
results['fdr corrected pval'] = multipletests(results['pval'], method = 'fdr_bh')[1]
results = results.sort_values(['rho'], ascending=False)
results.head(10)

,vars,otu,taxonomy,rho,pval,fdr corrected pval
557,OHVD3,557,k__Bacteria; p__Firmicutes; c__Erysipelotrichi...,0.190356,0.000005,0.023402
509,OHVD3,509,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.154249,0.000227,0.506881
369,OHVD3,369,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.150416,0.000325,0.506881
389,OHVD3,389,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.138464,0.000947,0.750827
143,OHVD3,143,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.134115,0.001370,0.750827
1247,OHVD3,1247,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.133798,0.001407,0.750827
117,OHVD3,117,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.133036,0.001499,0.750827
1536,OHVD3,1536,k__Bacteria; p__Bacteroidetes; c__Bacteroidia;...,0.133027,0.001500,0.750827
246,OHVD3,246,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.130323,0.001873,0.750827
204,OHVD3,204,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.130044,0.001916,0.750827


In [53]:
results.taxonomy[0]

'k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__Ruminococcus; s__'

## correlation with ratio_activation

In [54]:
vars_vd = np.array(['OHV1D3', 'OHVD3', 'OHV24D3', 'ratio_activation', 'ratio_catabolism'])
otu_cols = dat.columns[64:4790]
results_spearman = []
results_pearson = []
for i in range(len(otu_cols)):
    tmp = dat[['ratio_activation', otu_cols[i]]].dropna(axis=0, how='any')
    rho1, pval1 = spearmanr(tmp['ratio_activation'], tmp[otu_cols[i]])
    tax = bt.loc['taxonomy'][otu_cols[i]]
    results_spearman.append([vars_vd[1], otu_cols[i], tax, rho1, pval1])

/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3162: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3163: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x)

In [59]:
results_spearman = pd.DataFrame(results_spearman, columns=['vars', 'otu', 'taxonomy', 'rho', 'pval']).dropna(axis=0, how='any')
results = results_spearman
results['fdr corrected pval'] = multipletests(results['pval'], method = 'fdr_bh')[1]
results = results.sort_values(['rho'], ascending=False)
results.head(20)

,vars,otu,taxonomy,rho,pval,fdr corrected pval
83,OHVD3,83,k__Bacteria; p__Actinobacteria; c__Coriobacter...,0.205432,0.000001,0.004804
758,OHVD3,758,k__Bacteria; p__Actinobacteria; c__Actinobacte...,0.186580,0.000009,0.021928
822,OHVD3,822,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.182982,0.000014,0.021928
538,OHVD3,538,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.176672,0.000028,0.022483
573,OHVD3,573,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.176468,0.000029,0.022483
50,OHVD3,50,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.176328,0.000029,0.022483
826,OHVD3,826,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.170989,0.000051,0.032006
330,OHVD3,330,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.169022,0.000062,0.032006
63,OHVD3,63,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.162141,0.000123,0.041616
801,OHVD3,801,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.160531,0.000144,0.041616


In [61]:
for i in range(17):
    print(results.taxonomy[i])

k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__Ruminococcus; s__
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__Faecalibacterium; s__prausnitzii
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__Faecalibacterium; s__prausnitzii
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__uniformis
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__; g__; s__
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Rikenellaceae; g__; s__
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnosp

In [ ]:
## additional otu
Lachnospira, Parabacteroides, 

## correlation with ratio_catabolism

In [62]:
vars_vd = np.array(['OHV1D3', 'OHVD3', 'OHV24D3', 'ratio_activation', 'ratio_catabolism'])
otu_cols = dat.columns[64:4790]
results_spearman = []
results_pearson = []
for i in range(len(otu_cols)):
    tmp = dat[['ratio_catabolism', otu_cols[i]]].dropna(axis=0, how='any')
    rho1, pval1 = spearmanr(tmp['ratio_catabolism'], tmp[otu_cols[i]])
    tax = bt.loc['taxonomy'][otu_cols[i]]
    results_spearman.append([vars_vd[1], otu_cols[i], tax, rho1, pval1])

/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3162: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/numpy/lib/function_base.py:3163: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/serene/miniconda3/envs/gneiss_v2/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x)

In [64]:
results_spearman = pd.DataFrame(results_spearman, columns=['vars', 'otu', 'taxonomy', 'rho', 'pval']).dropna(axis=0, how='any')
results = results_spearman
results['fdr corrected pval'] = multipletests(results['pval'], method = 'fdr_bh')[1]
results = results.sort_values(['rho'], ascending=False)
results.head(5)

,vars,otu,taxonomy,rho,pval,fdr corrected pval
557,OHVD3,557,k__Bacteria; p__Firmicutes; c__Erysipelotrichi...,0.181905,0.000016,0.074027
369,OHVD3,369,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.170074,0.000056,0.129377
204,OHVD3,204,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.164709,0.000096,0.148100
352,OHVD3,352,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.159012,0.000167,0.193697
246,OHVD3,246,k__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.156417,0.000213,0.195909
